Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

In [160]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Regression II

In this project, you'll continue working with the New York City rent dataset you used in the last module project.

## Directions

The tasks for this project are as follows:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function to engineer two new features.
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline mean absolute error for your dataset.
- **Task 6:** Build and train a `Linearregression` model.
- **Task 7:** Calculate the training and test mean absolute error for your model.
- **Task 8:** Calculate the training and test $R^2$ score for your model.
- **Stretch Goal:** Determine the three most important features for your linear regression model.

**Note**

You should limit yourself to the following libraries for this project:

- `matplotlib`
- `numpy`
- `pandas`
- `sklearn`

# I. Wrangle Data

In [161]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  train_test_split

In [162]:
def wrangle(filepath):
    df = pd.read_csv(filepath,
                     parse_dates = ['created'],
                     index_col = 'created')
    
    # Remove the most extreme 1% prices,
    # the most extreme .1% latitudes, &
    # the most extreme .1% longitudes
    df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
            (df['price'] <= np.percentile(df['price'], 99.5)) & 
            (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
            (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
            (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
            (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
    # drop na
    df = df.dropna()
    # df['interest_level'] = df['interest_level'].map({'low': 0, 'medium': 1, 'high': 2})
    # one hot encoding: convert categorical value into binary
    df = pd.get_dummies(df, columns=['interest_level']) 
    # drop high variance (high cardinality) object
    df = df.drop(columns=['display_address', 'street_address']) 
    # total room
    df['total_room'] = df['bathrooms'] + df['bedrooms'] 
    # are pet allow
    df.loc[(df['cats_allowed'] + df['dogs_allowed']) != 0, 'pet_allow'] = 1 #sum not equal to 0 means pet is allow 
    df.loc[(df['cats_allowed'] + df['dogs_allowed']) == 0, 'pet_allow'] = 1 #sum equal to 0 means no pet allow
    # number of perks
    df['total_features'] = df.select_dtypes('int').drop(columns=['bedrooms','price']).sum(axis=1)

    return df

filepath = DATA_PATH + 'apartments/renthop-nyc.csv'

**Task 1:** Add the following functionality to the above `wrangle` function.

- The `'created'` column will parsed as a `DateTime` object and set as the `index` of the DataFrame. 
- Rows with `NaN` values will be dropped.

Then use your modified function to import the `renthop-nyc.csv` file into a DataFrame named `df`.

In [163]:
df = wrangle(filepath)

In [164]:
df.head()

,bathrooms,bedrooms,description,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,interest_level_high,interest_level_low,interest_level_medium,total_room,pet_allow,total_features
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,40.7145,-73.9425,3000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.5,1.0,0
2016-06-12 12:19:27,1.0,2,,40.7947,-73.9667,5465,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.0,1.0,5
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",40.7388,-74.0018,2850,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2.0,1.0,3
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,40.7539,-73.9677,3275,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.0,1.0,2
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,40.8241,-73.9493,3350,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5.0,1.0,1


In [165]:
(df.isna()).sum().sum()

0

In [166]:
df.shape

(47260, 36)

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47260 entries, 2016-06-24 07:54:24 to 2016-04-12 02:48:07
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bathrooms              47260 non-null  float64
 1   bedrooms               47260 non-null  int64  
 2   description            47260 non-null  object 
 3   latitude               47260 non-null  float64
 4   longitude              47260 non-null  float64
 5   price                  47260 non-null  int64  
 6   elevator               47260 non-null  int64  
 7   cats_allowed           47260 non-null  int64  
 8   hardwood_floors        47260 non-null  int64  
 9   dogs_allowed           47260 non-null  int64  
 10  doorman                47260 non-null  int64  
 11  dishwasher             47260 non-null  int64  
 12  no_fee                 47260 non-null  int64  
 13  laundry_in_building    47260 non-null  int64  
 14  fitness_center     

**Task 2:** Using your `pandas` and dataviz skills decide on two features that you want to engineer for your dataset. Next, modify your `wrangle` function to add those features. 

**Note:** You can learn more about feature engineering [here](https://en.wikipedia.org/wiki/Feature_engineering). Here are some ideas for new features:

- Does the apartment have a description?
- Length of description.
- Total number of perks that apartment has.
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths).

In [168]:
# Conduct your exploratory data analysis here, 
# and then modify the function above.

In [169]:
sum(df['dogs_allowed'] == df['cats_allowed'])

45711

In [170]:
df.shape

(47260, 36)

In [171]:
df['description'].nunique()

37727

# II. Split Data

**Task 3:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'price'`.

**Note:** In contrast to the last module project, this time you should include _all_ the numerical features in your dataset.

In [172]:
X = df[df.select_dtypes(exclude='object').columns].drop(columns='price')
y = df['price']
X.head()

,bathrooms,bedrooms,latitude,longitude,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,interest_level_high,interest_level_low,interest_level_medium,total_room,pet_allow,total_features
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.5,1.0,0
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.0,1.0,5
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2.0,1.0,3
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2.0,1.0,2
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5.0,1.0,1


**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from April and May 2016. 
- Your test set should include data from June 2016.

In [173]:
cutoff = '2016-06-01 00:00:00'
mask = X.index < cutoff
X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [174]:
X_train.shape[0]+X_test.shape[0]

47260

# III. Establish Baseline

**Task 5:** Since this is a **regression** problem, you need to calculate the baseline mean absolute error for your model. First, calculate the mean of `y_train`. Next, create a list `y_pred` that has the same length as `y_train` and where every item in the list is the mean. Finally, use `mean_absolute_error` to calculate your baseline.

In [175]:
y_pred = [y_train.mean()] * len(y_train)
baseline_mae = mean_absolute_error(y_train, y_pred)
print('Baseline MAE:', baseline_mae)

Baseline MAE: 1202.398300781848


# IV. Build Model

**Task 6:** Build and train a `LinearRegression` model named `model` using your feature matrix `X_train` and your target vector `y_train`.

In [176]:
# Step 1: Import predictor class
#import above

# Step 2: Instantiate predictor
model = LinearRegression()

# Step 3: Fit predictor on the (training) data
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# V. Check Metrics

**Task 7:** Calculate the training and test mean absolute error for your model.

In [177]:
train_mae = mean_absolute_error(y_train, model.predict(X_train))
test_mae = mean_absolute_error(y_test, model.predict(X_test))

print('Train MAE:', train_mae)
print('Test MAE:', test_mae)

Train MAE: 673.0105150524262
Test MAE: 675.5829002016359


**Task 8:** Calculate the training and test $R^2$ score for your model.

In [178]:
train_r2 = model.score(X_train, y_train)
test_r2 = model.score(X_test, y_test)

print('Train R^2:', train_r2)
print('Test R^2:', test_r2)

Train R^2: 0.6369282155858065
Test R^2: 0.6490901932805453


# VI. Communicate Results

**Stretch Goal:** What are the three most influential coefficients in your linear model? You should consider the _absolute value_ of each coefficient, so that it doesn't matter if it's positive or negative.

In [179]:
pd.Series(model.coef_, index=X_train.columns).abs().sort_values(ascending=False)

total_room               9.020434e+14
bedrooms                 9.020434e+14
bathrooms                9.020434e+14
longitude                1.307766e+04
latitude                 1.215304e+03
laundry_in_unit          4.427211e+02
interest_level_low       4.076039e+02
doorman                  3.839263e+02
high_speed_internet      3.252654e+02
interest_level_high      3.079844e+02
dining_room              2.259478e+02
roof_deck                1.953894e+02
hardwood_floors          1.777213e+02
common_outdoor_space     1.727248e+02
exclusive                1.712687e+02
new_construction         1.692831e+02
laundry_in_building      1.477869e+02
terrace                  1.408023e+02
no_fee                   1.383484e+02
wheelchair_access        1.377686e+02
pre-war                  1.273863e+02
outdoor_space            1.163456e+02
interest_level_medium    9.932551e+01
loft                     9.687128e+01
elevator                 9.678763e+01
garden_patio             8.166227e+01
balcony     

### The most important features are total number of rooms, bedroom, bathroom, location(longitude, latitude)